In [1]:
import math
import re
from collections import defaultdict

import pandas as pd
from gensim.models import KeyedVectors
from scipy.spatial import distance

from sentence2vec import Word, Sentence, sentence_to_vec



In [2]:
# euclidean distance between two vectors
def l2_dist(v1, v2):
    sum = 0.0
    if len(v1) == len(v2):
        for i in range(len(v1)):
            delta = v1[i] - v2[i]
            sum += delta * delta
        return math.sqrt(sum)


In [3]:

vectors = KeyedVectors.load_word2vec_format(
    "C:/Users/Asus/PycharmProjects/Mimic_3/extract_noun_phrases/bio_embedding_intrinsic", binary=True)

embedding_size = 200

In [6]:
embedding_size = 200
sentences = []
notes = pd.read_csv("C:/Users/Asus/Desktop/notes_physician_texttrim_np.csv")
notes.rename(columns={'TEXT_TRIM_NOUN_PHRASES':'NOTE'}, inplace=True)

In [7]:
notes.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,ISERROR,TEXT,TEXT_TRIM,NOTE
0,0,54610,100003.0,2150-04-17,2150-04-17 20:48:00,2150-04-17 20:48:38,Physician,Physician Attending Admission Note - MICU,NaN,Chief Complaint: GIB/HOTN\n I saw and exami...,Assessment and Plan\n HOTN: [**Month (only...,"[Assessment and Plan\n HOTN, **Month, *, GIB..."
1,1,54610,100003.0,2150-04-18,2150-04-18 02:47:00,2150-04-18 14:53:40,Physician,Physician Resident Admission Note,NaN,"Chief Complaint: coffee ground emesis, light ...",Assessment and Plan\n Mr. [**Known lastname ...,"[Assessment, Plan, [**Known lastname, a 59 yo ..."
2,2,54610,100003.0,2150-04-18,2150-04-18 08:41:00,2150-04-18 08:42:05,Physician,Physician Attending Progress Note,NaN,Chief Complaint:\n I saw and examined the pa...,Assessment and Plan\n 59 yo M with HCV cirho...,"[HCV cirhossis, I/II, HoTN, mental status chan..."
3,3,54610,100003.0,2150-04-19,2150-04-19 06:28:00,2150-04-19 09:53:50,Physician,Physician Resident Progress Note,NaN,Chief Complaint:\n 24 Hour Events:\n CALLED ...,"Assessment and Plan\n GASTROINTESTINAL BLEED, ...","[Assessment and Plan\n GASTROINTESTINAL BLEED,..."
4,4,54610,100003.0,2150-04-19,2150-04-19 09:54:00,2150-04-19 09:54:09,Physician,Physician Attending Progress Note,NaN,Chief Complaint:\n HPI:\n 24 Hour Events:\...,Assessment and Plan\n 59 yo M HCV cirrhosis ...,[Assessment and Plan\n 59 yo M HCV cirrhosis...


In [8]:
notes['NOTE'] = notes['NOTE'].apply(lambda x: re.sub(r'[,\[\]#\'"]', '', x))
notes['NOTE'] = notes['NOTE'].apply(lambda x: re.sub(' +', ' ', x))
notes['NOTE'] = notes['NOTE'].apply(lambda x: re.sub(r'[-/()+*:><]', ' ', x))
notes['NOTE'] = notes['NOTE'].apply(lambda x: x.lower())
sentences = list(notes.NOTE)
diagnoses_list = pd.read_csv('C:/Users/Asus/PycharmProjects/Mimic_3/extract_noun_phrases/D_ICD_DIAGNOSES.csv')
diagnoses_list['LONG_TITLE'] = diagnoses_list['LONG_TITLE'].apply(lambda x: re.sub(r'[,\[\]#\'"]', '', x))
diagnoses_list['LONG_TITLE'] = diagnoses_list['LONG_TITLE'].apply(lambda x: x.replace('-', ' '))
diagnoses_list['LONG_TITLE'] = diagnoses_list['LONG_TITLE'].apply(lambda x: x.lower())
diag = list(diagnoses_list.LONG_TITLE)